In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    print("bu")
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
import json
import torch
from google.colab import drive
from datasets import Dataset, Features, Sequence, Value, ClassLabel
from transformers import TrainingArguments, AutoTokenizer, AutoConfig
import os

drive.mount('/content/drive')
FOLDERNAME = '/content/drive/MyDrive/NLPproje/'
if not os.path.exists(FOLDERNAME): os.makedirs(FOLDERNAME)
os.chdir(FOLDERNAME)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cihaz: {device}")

with open('supplements_spans_ents.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

all_labels = set(ent["label"] for item in raw_data for ent in item["entities"])

print("Labels:", all_labels)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cihaz: cuda
Labels: {'HASTALIK', 'MARKA', 'TAVSİYE_EDEN', 'SAĞLIK_ŞİKAYETLERİ', 'DOZ', 'ÜRÜN_DİĞER', 'TAT_KOKU', 'KULLANICI', 'BİYOMOLEKÜL', 'İÇERİK', 'MARKA_DİĞER', 'YAN_ETKİ', 'ETKİ', 'KULLANICI_DEMOGRAFİSİ'}


In [ ]:

from unsloth import FastLanguageModel

label_map = {
    "MARKA": "URUN_MARKA", "MARKA_DİĞER": "URUN_MARKA", "ÜRÜN_DİĞER": "URUN_MARKA",
    "İÇERİK": "BILESEN", "BİYOMOLEKÜL": "BILESEN",
    "HASTALIK": "SAGLIK_DURUMU", "SAĞLIK_ŞİKAYETLERİ": "SAGLIK_DURUMU",
    "ETKİ": "ETKI_DENEYIM", "YAN_ETKİ": "ETKI_DENEYIM",
    "KULLANICI": "KULLANICI_BILGISI", "KULLANICI_DEMOGRAFİSİ": "KULLANICI_BILGISI",
    "TAVSİYE_EDEN": "TAVSİYE_EDEN",
    "DOZ":"DOZ",
    "TAT_KOKU":"TAT_KOKU"
}

def create_xml_text(text, entities):
    entities = sorted(entities, key=lambda x: x["start"], reverse=True)
    xml_text = text
    for ent in entities:
        label = label_map.get(ent["label"], ent["label"])
        s, e = ent["start"], ent["end"]
        xml_text = xml_text[:s] + f"<{label}>" + xml_text[s:e] + f"</{label}>" + xml_text[e:]
    return xml_text


system_prompt = """Sen uzman bir takviye edici gıda veri analistisin. Kullanıcı yorumlarını analiz edip, varlıkları (entity) XML etiketleri içine alarak yapılandırman gerekiyor.

ETİKETLEME KURALLARI:
1. URUN_MARKA: Markalar veya ürün adları (balık yağı, Ocean).
2. BILESEN: Maddeler (Magnezyum, D3 vitamini).
3. DOZ: Miktar ve sıklık (200mg, günde 1 kez).
4. SAGLIK_DURUMU: Hastalıklar veya şikayetler (kabızlık, PCOS, halsizlik).
5. ETKI_DENEYIM: Kullanım sonrası tüm sonuçlar (iyi geldi, ishal yaptı, baş ağrımı arttırıyor).
6. KULLANICI_BILGISI: Kullanıcı tanımları (sporcular, 30 yaşındaki kadın).
7. TAVSİYE_EDEN: Öneren kişi (doktor, psikolog )
8. TAT_KOKU: Kullanılan ürünün tadı veya kokusu

KRİTİK TALİMATLAR:
- İÇ İÇE (NESTED) ETİKETLEME: Eğer bir durumun sonucu (ETKI_DENEYIM), içinde bir hastalığı (SAGLIK_DURUMU) barındırıyorsa, her ikisini de etiketle.
  Örn: 'kabızlığıma iyi geldi' ifadesinde 'kabızlık' hastalık, ifadenin tamamı deneyimdir.
- YAZIM YANLIŞLARI: Kelimeler yanlış yazılsa bile (örn: 'kabızlk', 'vitmain') orijinal karakterleri bozmadan etiketle.
- METİN SADAKATİ: Orijinal metindeki tek bir boşluğu veya karakteri değiştirme.

ÖRNEK (Nested & Hiyerarşik):
Girdi: "Bu magnezyum kabızlığıma iyi geldi."
Çıktı: "Bu <BILESEN>magnezyum</BILESEN> <ETKI_DENEYIM><SAGLIK_DURUMU>kabızlığıma</SAGLIK_DURUMU> iyi geldi</ETKI_DENEYIM>."

ÖRNEK (Marka & Bileşen):
Girdi: "Solgar Magnezyum kullandım."
Çıktı: "<URUN_MARKA>Solgar <BILESEN>Magnezyum</BILESEN></URUN_MARKA> kullandım."
"""

formatted_data = []
for item in raw_data:
    xml_out = create_xml_text(item["text"], item.get("entities", []))
    formatted_data.append({
        "instruction": system_prompt,
        "input": item["text"],
        "output": xml_out
    })

dataset = Dataset.from_list(formatted_data)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/tmp/ipython-input-3621776395.py:1: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
formatted_data[0]

{'instruction': 'Sen uzman bir takviye edici gıda veri analistisin. Kullanıcı yorumlarını analiz edip, varlıkları (entity) XML etiketleri içine alarak yapılandırman gerekiyor.\n\nETİKETLEME KURALLARI:\n1. URUN_MARKA: Markalar veya ürün adları (balık yağı, Ocean).\n2. BILESEN: Maddeler (Magnezyum, D3 vitamini).\n3. DOZ: Miktar ve sıklık (200mg, günde 1 kez).\n4. SAGLIK_DURUMU: Hastalıklar veya şikayetler (kabızlık, PCOS, halsizlik).\n5. ETKI_DENEYIM: Kullanım sonrası tüm sonuçlar (iyi geldi, ishal yaptı, baş ağrımı arttırıyor).\n6. KULLANICI_BILGISI: Kullanıcı tanımları (sporcular, 30 yaşındaki kadın).\n7. TAVSİYE_EDEN: Öneren kişi (doktor, psikolog )\n8. TAT_KOKU: Kullanılan ürünün tadı veya kokusu\n\nKRİTİK TALİMATLAR:\n- İÇ İÇE (NESTED) ETİKETLEME: Eğer bir durumun sonucu (ETKI_DENEYIM), içinde bir hastalığı (SAGLIK_DURUMU) barındırıyorsa, her ikisini de etiketle.\n  Örn: \'kabızlığıma iyi geldi\' ifadesinde \'kabızlık\' hastalık, ifadenin tamamı deneyimdir.\n- YAZIM YANLIŞLARI: Keli

In [ ]:


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = 4096,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
)

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        convo = [
            {"from": "system", "value": instruction},
            {"from": "human",  "value": input_text},
            {"from": "gpt",    "value": output},
        ]
        texts.append(tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False))
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

first_split = dataset.train_test_split(test_size=0.2, seed=3407)
train_data = first_split["train"]
remainder = first_split["test"]

second_split = remainder.train_test_split(test_size=0.5, seed=3407)
eval_data = second_split["train"]
test_data = second_split["test"]

print(f"Eğitim seti:   {len(train_data)}")
print(f"Doğrulama seti: {len(eval_data)} ")
print(f"Final Test seti: {len(test_data)}")

Map:   0%|          | 0/2472 [00:00<?, ? examples/s]

Eğitim seti:   1977
Doğrulama seti: 247 
Final Test seti: 248


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    eval_dataset = eval_data,
    dataset_text_field = "text",
    max_seq_length = 4096,
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        per_device_eval_batch_size = 8,
        gradient_accumulation_steps = 2,
        warmup_steps = 10,
        max_steps = 100,
        learning_rate = 2e-4,
        bf16 = True,
        logging_steps = 1,

        eval_strategy = "steps",
        eval_steps = 20,
        save_strategy = "steps",
        save_steps = 20,
        load_best_model_at_end = True,

        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        output_dir = "outputs",
        report_to = "none"
    ),
)

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer.train()


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1977 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/247 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/1977 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/247 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,977 | Num Epochs = 2 | Total steps = 100
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 2 x 1) = 32
 "-____-"     Trainable parameters = 83,886,080 of 8,114,147,328 (1.03% trained)


Step,Training Loss,Validation Loss
20,0.148600,0.153921
40,0.098600,0.119648
60,0.111100,0.107614
80,0.076200,0.103649
100,0.088900,0.102327


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=100, training_loss=0.13351879253983498, metrics={'train_runtime': 620.5506, 'train_samples_per_second': 5.157, 'train_steps_per_second': 0.161, 'total_flos': 1.4472834989602406e+17, 'train_loss': 0.13351879253983498, 'epoch': 1.6129032258064515})

In [ ]:

def test_func(model,test_data):
  import random
  import re
  import torch

  random_test_samples = random.sample(list(test_data), 5)

  FastLanguageModel.for_inference(model)

  print("-" * 70)
  print(f"{'TEST':^70}")
  print("-" * 70)

  for i, sample in enumerate(random_test_samples, 1):
      input_text = sample["input"]

      real_answer = sample.get("output", "")

      print(f"Test {i}: {input_text}")

      print("\n[ORİJİNAL VARLIKLAR ]")

      real_entities = re.findall(r"<(\w+)>(.*?)</\1>", real_answer)

      if not real_entities:
          print("   Orijinal veride etiket bulunmuyor.")
      else:
          for label, ent_text in real_entities:
              print(f"    {ent_text} => {label}")

      messages = [
          {"from": "system", "value": sample["instruction"]},
          {"from": "human", "value": input_text}
      ]

      inputs = tokenizer.apply_chat_template(
          messages,
          add_generation_prompt=True,
          return_tensors="pt"
      ).to("cuda")

      with torch.no_grad():
          outputs = model.generate(input_ids=inputs, max_new_tokens=256, use_cache=True)

      full_response = tokenizer.batch_decode(outputs)[0]
      prediction = full_response.split("<|start_header_id|>assistant<|end_header_id|>\n\n")[-1].replace("<|eot_id|>", "").strip()

      print(f"\nMODEL XML ÇIKTISI:\n{prediction}")



      print("\n[MODEL TAHMİNİ]")
      pred_entities = re.findall(r"<(\w+)>(.*?)</\1>", prediction)

      if not pred_entities:
          print("    Herhangi bir entity tespit edilemedi.")
      else:
          for label, ent_text in pred_entities:
            print(f"     {ent_text} => {label}")

      print("\n" + "-" * 70)

In [ ]:
test_func(model,test_data)

In [ ]:
import os

hedef = "llama3_ner_v2"
FINAL_PATH = os.path.join(FOLDERNAME, hedef)

os.makedirs(FINAL_PATH, exist_ok=True)

model.save_pretrained(FINAL_PATH)
tokenizer.save_pretrained(FINAL_PATH)

print(f"Model Konumu: {FINAL_PATH}")

Model Konumu: /content/drive/MyDrive/NLPproje/llama3_ner_v2


In [ ]:
import torch
from unsloth import FastLanguageModel
import os


hedef = "llama3_ner_v2"
FINAL_PATH = os.path.join(FOLDERNAME, hedef)


print(f"Model şuradan yükleniyor: {FINAL_PATH}")

model_from_drive, tokenizer = FastLanguageModel.from_pretrained(
    model_name = FINAL_PATH,
    max_seq_length = 4096,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model_from_drive)

print("Model yüklendi")




In [ ]:
test_func(model_from_drive,test_data)

----------------------------------------------------------------------
                                 TEST                                 
----------------------------------------------------------------------
Test 1: Doktorumun tavsiyesi üzerine 1 ay önce başaldım burdan 2. Kutuyu aldım saç dökülmem asırı azaldı ve sureklı kırılan yaprak gibi ince olan tırnaklarım sertleşti tavsiye ederim.. yanında nail balsam kremide kullanabilirsinizi oda cok etkilş

[ORİJİNAL VARLIKLAR ]
    Doktorumun => TAVSİYE_EDEN
    saç dökülmem asırı azaldı ve sureklı kırılan yaprak gibi ince olan tırnaklarım sertleşti => ETKI_DENEYIM
    nail balsam => URUN_MARKA

MODEL XML ÇIKTISI:
<TAVSİYE_EDEN>Doktorumun</TAVSİYE_EDEN> tavsiyesi üzerine 1 ay önce başaldım burdan 2. Kutuyu aldım <ETKI_DENEYIM><SAGLIK_DURUMU>saç dökülmem</SAGLIK_DURUMU> asırı azaldı ve sureklı kırılan yaprak gibi ince olan tırnaklarım sertleşti</ETKI_DENEYIM> tavsiye ederim.. yanında <URUN_MARKA>nail balsam kremide</URUN_MARKA> kullanab